**50 nuances d'IA : Introduction**



In [0]:
import keras
from keras.datasets import cifar10
from matplotlib import pyplot as plt
import numpy as np
from collections import defaultdict
import math  
%matplotlib inline 


(x_train, y_train), (x_test, y_test) = cifar10.load_data() #on charge le jeu de données

Pour vous présenter les notions principales du Machine Learning, nous allons vous introduire deux algorithmes de base : KNN et K-MEAN.

Ils seront appliqués au jeu de données CIFAR 10, jeu de données de 50 000 images appartenant à 10 classes d'images différentes.

In [0]:
# Observons les dimensions du jeu de données
print("Images du jeu d'entrainement "+str(x_train.shape));
print("Classes du jeu d'entrainement "+str(y_train.shape));

In [0]:
# Visualisons un exemple et sa classe
plt.imshow(x_train[0])
plt.show()

print(y_train[0])

In [0]:
#visualisation du jeu de données
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

K-NN (K Nearest Neighbor ou méthode des K plus proches voisins) est un algorithme consistant à trouver, dans le jeu de données d'entrainement, les K images ressemblant le plus à l'image dont nous souhaitons trouver la classe. Pour calculer la ressemblance entre deux images on peut considérer simplement leur distance euclidienne (norme L2). Sur les K images trouvées, nous regarderons ensuite quelle classe est la plus présente. On pourra ainsi décider de la classe de notre image de test.


In [0]:
# Nous redimensionons les images en les applatissant afin de faciliter leur manipulation
x_train = x_train.reshape(50000,32*32*3)
x_test = x_test.reshape(10000,32*32*3)

In [0]:
# --- Méthode brute --- Code d'un 1NN
nearest = []
for id_image, image in enumerate(x_test):
  # On initialise les variables
  min_d = None
  min_k = None
  for id_comp_image, comp_image in enumerate(x_train):
    # On calcule la distance euclidienne entre les images (Norme euclidienne sur des dans R^n)
    d = np.linalg.norm(image - comp_image)
    if (min_d is None) or (d < min_d):
      min_d = d
      min_k = id_comp_image
  print(y_train[min_k])
  nearest.append(y_train[min_k])

Cependant le code au dessus est pas très beau et le réimplémenter est un peu pénible.
Afin de simplifier la vie de tout le monde, nous allons utiliser une bibliothèque du nom de **sickit learn**.
Cette bibliothèque est une boite à outils remplie de beaucoup de fonctions très pratiques et d'algorithmes d'apprentissages prêts à l'utilisation. On vous laisse chercher [ici](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) comment l'utiliser
**Nous vous déconseillons de faire l'entrainement sur les 50 000 éléments de x_train et le test sur les 10 000 éléments de x_test car cela vous prendrait trop de temps pour tester...**

In [0]:
from sklearn.neighbors import KNeighborsClassifier #on importe la bibliothèque
#Regardez la fonction fit et la fonction score...
#TODO

La fonction score nous a permis de mesure la précision (accuracy) de l'algorithme KNN sur une partie de notre
jeu de données.
Nous avons obtenu 0.29 ce qui veut dire que sur les 100 images testées, seules 29% étaient correctes.
De plus, vous avez pu remarquer que le temps d'exécution était plutôt long. Imaginez le temps que cela mettrait si l'on voulait tester l'intégralité de notre jeu de test
qui avait 10 000 exemples....
Ici, nous avons testé pour k = 7. Mais quelle est la valeur optimale de **k**? k est ce que l'on apelle un **hyperparamètre**. C'est une valeur à configurer avant l'entrainement de notre modèle sur le jeu de données.
A vous de la trouver...


In [0]:
#mettre ici le code de recherche du meilleur k
#TODO


La méthode K-MEAN (ou méthode des K-moyennes) est un algorithme de partionnement de données (*clustering* en anglais). C'est l'un des algorithmes les plus fondamentaux en apprentissage non supervisé. L'algorithme consiste  à partionner des données pour tenter d'en dégager des classes. Dans notre cas, appliqué aux images de CIFAR-10, cela  revient a classer les images tel que cela est déjà fait mais en utilisant juste les données brutes (c'est pour cela que l'on parle d'apprentissage *non supervisé*). L'idée générale derrière la méthode est de regrouper les données en fonction de leurs ressemblances, i.e. de leur distance. L'algorithme fonctionne ainsi: on commence en considérant K données aléatoires, elles sont chacunes représentantes d'une classe ; à chaque itération, on va partionner les données en fonction de la ressemblance avec les K images types de départ : on regroupe dans la classe K toutes les images étant plus proches de la K-ème image type ; on calcule ensuite la moyenne des classes obtenues et l'on remplace l'image type de chacune des classes obtenues par cette moyenne.

In [0]:
K_VALUE = 10

min_val = 1
# On initialise les K representants de chaque classe
K_mean = [255 * np.random.rand(32*32*3) for _ in range(10)]
# Valeur précédente de ces representants
K_save = [255 * np.random.rand(32*32*3) for _ in range(10)]

def nearest_K(image):
  """
  Retourne la classe K la plus proche de image
  """
  min_d = None
  min_k = None
  for id_K, K_point in enumerate(K_mean):
    d = np.linalg.norm(image - K_point)
    if (min_d is None) or (d < min_d):
      min_d = d
      min_k = id_K
  return min_k

def mean_point(k, tab):
  """
  Retourne barycentre des points (indicés) de tab
  """
  if tab != []:
    mean = 0
    for id in tab:
      mean += x_train[id] / len(tab)
    K_mean[k] = mean
    
def stop_convergence():
  """
  Evalue si l'on arrete les itérations
  """
  for k in range(10):
    if not(np.array_equal(K_mean[k], K_save[k])):
      return True
  return False

#KMEAN
iteration = 0
while stop_convergence():
  iteration += 1
  K_nearest = [[] for _ in range(10)]
  for id_image, image in enumerate(x_train):
    K_nearest[nearest_K(image)].append(id_image)
  for k in range(10):
    K_save[k] = K_mean[k]
    mean_point(k, K_nearest[k])
  print(iteration)
    


In [0]:
from sklearn.cluster import KMeans
#TODO